In [1]:
"""import sys

#!{sys.executable} -m conda create -n py36 python=3.6 anaconda -y
!{sys.executable} -m pip install install-jdk
!{sys.executable} -m pip install requests
!{sys.executable} -m pip install tabulate
!{sys.executable} -m pip install "colorama>=0.3.8"
!{sys.executable} -m pip install future
!{sys.executable} -m pip uninstall h2o
!{sys.executable} -m pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o"""

'import sys\n\n#!{sys.executable} -m conda create -n py36 python=3.6 anaconda -y\n!{sys.executable} -m pip install install-jdk\n!{sys.executable} -m pip install requests\n!{sys.executable} -m pip install tabulate\n!{sys.executable} -m pip install "colorama>=0.3.8"\n!{sys.executable} -m pip install future\n!{sys.executable} -m pip uninstall h2o\n!{sys.executable} -m pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o'

In [1]:
import h2o
import jdk
from h2o.automl import H2OAutoML
import numpy as np
import matplotlib as plt
import pandas as pd
%matplotlib inline

In [2]:
import os
import h2o

startup  = '/home/h2o/bin/aquarium_startup'
shutdown = '/home/h2o/bin/aquarium_stop'

if os.path.exists(startup):
    os.system(startup)
    local_url = 'http://localhost:54321/h2o'
    aquarium = True
else:
    local_url = 'http://localhost:54321'
    aquarium = False

In [3]:
h2o.init(url=local_url)

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "14" 2020-03-17; OpenJDK Runtime Environment (build 14+36-1461); OpenJDK 64-Bit Server VM (build 14+36-1461, mixed mode, sharing)
  Starting server from /opt/conda/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp8c86cr0o
  JVM stdout: /tmp/tmp8c86cr0o/h2o_jovyan_started_from_python.out
  JVM stderr: /tmp/tmp8c86cr0o/h2o_jovyan_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.0.5
H2O_cluster_version_age:,26 days
H2O_cluster_name:,H2O_from_python_jovyan_qh2gha
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,2.322 Gb
H2O_cluster_total_cores:,6
H2O_cluster_allowed_cores:,6
H2O_cluster_status:,"accepting new members, healthy"


In [4]:
array_var_obj = ['wardsPlaced - TOPB', 'wardsPlaced - JGB', 'wardsPlaced - MIDB', 'wardsPlaced - ADCB', 'wardsPlaced - SUPPB', 'wardsPlaced - TOPR', 'wardsPlaced - JGR', 'wardsPlaced - MIDR', 'wardsPlaced - ADCR', 'wardsPlaced - SUPPR']
cat = ['baron','championId - TOPB', 'championId - SUPPR', 'championId - JGB', 'championId - MIDB', 'championId - ADCB', 'championId - SUPPB', 'championId - TOPR', 'championId - JGR', 'championId - MIDR', 'championId - ADCR','dead - TOPB', 'dead - SUPPR', 'dead - JGB', 'dead - MIDB', 'dead - ADCB', 'dead - SUPPB', 'dead - TOPR', 'dead - JGR', 'dead - MIDR', 'dead - ADCR', 'dragon','item0 - TOPB', 'item0 - SUPPR', 'item0 - JGB', 'item0 - MIDB', 'item0 - ADCB', 'item0 - SUPPB', 'item0 - TOPR', 'item0 - JGR', 'item0 - MIDR', 'item0 - ADCR', 'item1 - TOPB', 'item1 - SUPPR', 'item1 - JGB', 'item1 - MIDB', 'item1 - ADCB', 'item1 - SUPPB', 'item1 - TOPR', 'item1 - JGR', 'item1 - MIDR', 'item1 - ADCR', 'item2 - TOPB', 'item2 - SUPPR', 'item2 - JGB', 'item2 - MIDB', 'item2 - ADCB', 'item2 - SUPPB', 'item2 - TOPR', 'item2 - JGR', 'item2 - MIDR', 'item2 - ADCR','rift','wardsKilled - TOPB', 'wardsKilled - SUPPR', 'wardsKilled - JGB', 'wardsKilled - MIDB', 'wardsKilled - ADCB', 'wardsKilled - SUPPB', 'wardsKilled - TOPR', 'wardsKilled - JGR', 'wardsKilled - MIDR', 'wardsKilled - ADCR']
clean = {}
WOE = {}
WOESUPPR = {}

In [5]:
for var_obj in array_var_obj:
    print(var_obj)
    dfClean = h2o.import_file('FinalDataframe'+ var_obj +'.csv')
    dfClean = dfClean.drop("C1")
    
    dfClean[cat] = dfClean[cat].asfactor()
    #dfClean[var_obj] = dfClean[var_obj].asfactor()
    trainClean, testClean = dfClean.split_frame([0.75])
    
    yClean = var_obj
    ignoreClean = [var_obj] 
    xClean = list(set(trainClean.names) - set(ignoreClean))
    
    amlClean = H2OAutoML(max_runtime_secs=3600, project_name='clean_'+var_obj.replace(" ",""), balance_classes=True)
    amlClean.train(x=xClean, y=yClean, training_frame=trainClean)
    clean[var_obj] = amlClean
    """
    dfWOE = h2o.import_file('FinalDataframeWOE'+ var_obj +'.csv')
    dfWOE = dfWOE.drop("C1")
    
    #dfWOE[dfWOE.columns] = dfWOE[dfWOE.columns].asfactor()
    trainWOE, testWOE = dfWOE.split_frame([0.75])
    
    yWOE = var_obj
    ignoreWOE = [var_obj] 
    xWOE = list(set(trainWOE.names) - set(ignoreWOE))
    
    amlWOE = H2OAutoML(max_runtime_secs=1250, project_name='WOE_'+var_obj.replace(" ",""), balance_classes=True)
    amlWOE.train(x=xWOE, y=yWOE, training_frame=trainWOE)
    WOE[var_obj] = amlWOE
    
    dfWOESUPPR = h2o.import_file('SUPPRFinalDataframeWOE'+ var_obj +'.csv')
    dfWOESUPPR = dfWOESUPPR.drop("C1")
    
    #dfWOESUPPR[dfWOESUPPR.columns] = dfWOESUPPR[dfWOESUPPR.columns].asfactor()
    trainWOESUPPR, testWOESUPPR = dfWOESUPPR.split_frame([0.75])
    
    yWOESUPPR = var_obj
    ignoreWOESUPPR = [var_obj] 
    xWOESUPPR = list(set(trainWOESUPPR.names) - set(ignoreWOESUPPR))
    
    amlWOESUPPR = H2OAutoML(max_runtime_secs=1250, project_name='WOESUPPR_'+var_obj.replace(" ",""), balance_classes=True)
    amlWOESUPPR.train(x=xWOESUPPR, y=yWOESUPPR, training_frame=trainWOESUPPR)
    WOESUPPR[var_obj] = amlWOESUPPR"""
    
    print("-------------------------------------------------------------------------")

wardsPlaced - TOPB
Parse progress: |█████████████████████████████████████████████████████████| 100%
AutoML progress: |████████████████████████████████████████████████████████| 100%
-------------------------------------------------------------------------
wardsPlaced - JGB
Parse progress: |█████████████████████████████████████████████████████████| 100%
AutoML progress: |████████████████████████████████████████████████████████| 100%
-------------------------------------------------------------------------
wardsPlaced - MIDB
Parse progress: |█████████████████████████████████████████████████████████| 100%
AutoML progress: |████████████████████████████████████████████████████████| 100%
-------------------------------------------------------------------------
wardsPlaced - ADCB
Parse progress: |█████████████████████████████████████████████████████████| 100%
AutoML progress: |████████████████████████████████████████████████████████| 100%
-------------------------------------------------------

In [1]:
for var_obj in array_var_obj:
    pd = clean[var_obj].leaderboard.as_data_frame()
    #mod_ids = pd['model_id'].tolist()
    #mod_ids = clean[var_obj].leaderboard["model_id"].tolist()
    for i in range(0,4):

        aml1 = h2o.get_model(clean[var_obj].leaderboard[i, 0]) # get model object in environment
        h2o.save_model(model = aml1, path = "clean/"+var_obj+"/",force=True) # pass that model object to h2o.saveModel as an argument


NameError: name 'array_var_obj' is not defined

In [36]:
saved_model = h2o.load_model("clean/wardsPlaced - MIDB/StackedEnsemble_AllModels_AutoML_20210412_081313")

H2OResponseError: Server error water.exceptions.H2OIllegalArgumentException:
  Error: Illegal argument: dir of function: importModel: water.api.FSIOException: FS IO Failure: 
 accessed path : file:/home/jovyan/work/clean/wardsPlaced%20-%20MIDB/StackedEnsemble_AllModels_AutoML_20210412_081313 msg: File not found
  Request: POST /99/Models.bin/
    data: {'dir': 'clean/wardsPlaced - MIDB/StackedEnsemble_AllModels_AutoML_20210412_081313'}


In [35]:
saved_model

Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_AllModels_AutoML_20210412_081313

No model summary for this model

ModelMetricsBinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.16469624674961691
RMSE: 0.40582785359018536
LogLoss: 0.5034271920752278
Null degrees of freedom: 6124
Residual degrees of freedom: 6111
Null deviance: 7999.758632191014
Residual deviance: 6166.98310292154
AIC: 6194.98310292154
AUC: 0.8526553007830283
AUCPR: 0.7692558780071961
Gini: 0.7053106015660566

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.3430149429524226: 


,,False,True,Error,Rate
0,False,2681.0,1243.0,0.3168,(1243.0/3924.0)
1,True,365.0,1836.0,0.1658,(365.0/2201.0)
2,Total,3046.0,3079.0,0.2625,(1608.0/6125.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.343015,0.695455,230.0
1,max f2,0.271834,0.812129,279.0
2,max f0point5,0.489901,0.709622,135.0
3,max accuracy,0.453160,0.779102,157.0
4,max precision,0.796572,1.000000,0.0
5,max recall,0.115707,1.000000,381.0
6,max specificity,0.796572,1.000000,0.0
7,max absolute_mcc,0.411304,0.508454,185.0
8,max min_per_class_accuracy,0.381565,0.757390,203.0
9,max mean_per_class_accuracy,0.358358,0.759279,219.0



Gains/Lift Table: Avg response rate: 35.93 %, avg score: 36.11 %


...

,,False,True,Error,Rate
0,False,1538.0,2386.0,0.6081,(2386.0/3924.0)
1,True,350.0,1851.0,0.159,(350.0/2201.0)
2,Total,1888.0,4237.0,0.4467,(2736.0/6125.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.267395,0.575023,280.0
1,max f2,0.141223,0.753151,369.0
2,max f0point5,0.401994,0.514862,182.0
3,max accuracy,0.467704,0.661388,137.0
4,max precision,0.740799,0.833333,3.0
5,max recall,0.104938,1.000000,387.0
6,max specificity,0.796548,0.999745,0.0
7,max absolute_mcc,0.381884,0.247079,195.0
8,max min_per_class_accuracy,0.363655,0.622579,208.0
9,max mean_per_class_accuracy,0.381884,0.627424,195.0



Gains/Lift Table: Avg response rate: 35.93 %, avg score: 35.93 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.010122,0.692863,1.930024,1.930024,0.693548,0.718155,0.693548,0.718155,0.019537,0.019537,93.002448,93.002448,0.014695
1,2,0.020082,0.663930,1.687944,1.809968,0.606557,0.679224,0.650407,0.698847,0.016811,0.036347,68.794363,80.996812,0.025389
2,3,0.030041,0.642797,1.733564,1.784638,0.622951,0.652878,0.641304,0.683608,0.017265,0.053612,73.356373,78.463841,0.036792
3,4,0.040000,0.624302,1.459843,1.703771,0.524590,0.633423,0.612245,0.671113,0.014539,0.068151,45.984314,70.377101,0.043941
4,5,0.050122,0.614349,1.615834,1.686012,0.580645,0.619676,0.605863,0.660725,0.016356,0.084507,61.583444,68.601184,0.053671
5,6,0.100082,0.564538,1.482355,1.584350,0.532680,0.589620,0.569331,0.625230,0.074057,0.158564,48.235502,58.434954,0.091286
6,7,0.150041,0.529027,1.500543,1.556445,0.539216,0.545713,0.559304,0.598753,0.074966,0.233530,50.054343,55.644457,0.130319
7,8,0.200000,0.498273,1.418696,1.522035,0.509804,0.514101,0.546939,0.577607,0.070877,0.304407,41.869560,52.203544,0.162970
8,9,0.300082,0.445961,1.339207,1.461059,0.481240,0.471344,0.525027,0.542167,0.134030,0.438437,33.920663,46.105935,0.215960
9,10,0.400000,0.396435,1.195888,1.394821,0.429739,0.420748,0.501224,0.511837,0.119491,0.557928,19.588764,39.482054,0.246511


In [33]:
clean['wardsPlaced - ADCB'].leader

Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_AllModels_AutoML_20210412_050847

No model summary for this model

ModelMetricsBinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.12131305551172855
RMSE: 0.3483002375992996
LogLoss: 0.40621280072388094
Null degrees of freedom: 6071
Residual degrees of freedom: 6061
Null deviance: 7774.7708749818685
Residual deviance: 4933.048251990809
AIC: 4955.048251990809
AUC: 0.9553730511069634
AUCPR: 0.91424440671428
Gini: 0.9107461022139267

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.38377797788658563: 


,,False,True,Error,Rate
0,False,3608.0,407.0,0.1014,(407.0/4015.0)
1,True,252.0,1805.0,0.1225,(252.0/2057.0)
2,Total,3860.0,2212.0,0.1085,(659.0/6072.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.383778,0.845631,212.0
1,max f2,0.331617,0.886035,240.0
2,max f0point5,0.455138,0.865720,175.0
3,max accuracy,0.414014,0.894269,194.0
4,max precision,0.897674,1.000000,0.0
5,max recall,0.173022,1.000000,350.0
6,max specificity,0.897674,1.000000,0.0
7,max absolute_mcc,0.383778,0.763314,212.0
8,max min_per_class_accuracy,0.373014,0.886924,218.0
9,max mean_per_class_accuracy,0.373014,0.889014,218.0



Gains/Lift Table: Avg response rate: 33.88 %, avg score: 34.85 %


...

,,False,True,Error,Rate
0,False,1107.0,2908.0,0.7243,(2908.0/4015.0)
1,True,253.0,1804.0,0.123,(253.0/2057.0)
2,Total,1360.0,4712.0,0.5206,(3161.0/6072.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.241721,0.533018,317.0
1,max f2,0.148735,0.729795,379.0
2,max f0point5,0.343471,0.468864,224.0
3,max accuracy,0.528930,0.681489,91.0
4,max precision,0.890376,1.000000,0.0
5,max recall,0.148735,1.000000,379.0
6,max specificity,0.890376,1.000000,0.0
7,max absolute_mcc,0.343471,0.201640,224.0
8,max min_per_class_accuracy,0.329886,0.604732,236.0
9,max mean_per_class_accuracy,0.325940,0.605802,240.0



Gains/Lift Table: Avg response rate: 33.88 %, avg score: 33.89 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.010046,0.737884,2.322784,2.322784,0.786885,0.798585,0.786885,0.798585,0.023335,0.023335,132.278426,132.278426,0.020097
1,2,0.020092,0.668278,1.935654,2.129219,0.655738,0.702005,0.721311,0.750295,0.019446,0.042781,93.565355,112.921890,0.034313
2,3,0.030138,0.621468,1.984045,2.080828,0.672131,0.641653,0.704918,0.714081,0.019932,0.062713,98.404488,108.082756,0.049263
3,4,0.040020,0.590925,1.672727,1.980062,0.566667,0.606586,0.670782,0.687539,0.016529,0.079242,67.272727,98.006206,0.059316
4,5,0.050066,0.568895,1.645306,1.912891,0.557377,0.580256,0.648026,0.666012,0.016529,0.095771,64.530551,91.289052,0.069120
5,6,0.100132,0.504107,1.592457,1.752674,0.539474,0.533774,0.593750,0.599893,0.079728,0.175498,59.245708,75.267380,0.113979
6,7,0.150033,0.467217,1.276222,1.594205,0.432343,0.485271,0.540066,0.561769,0.063685,0.239183,27.622174,59.420511,0.134825
7,8,0.200099,0.438202,1.320574,1.525741,0.447368,0.451621,0.516872,0.534210,0.066116,0.305299,32.057416,52.574107,0.159097
8,9,0.300066,0.390337,1.103912,1.385209,0.373970,0.412377,0.469265,0.493621,0.110355,0.415654,10.391247,38.520871,0.174807
9,10,0.400033,0.350719,1.186584,1.335573,0.401977,0.370082,0.452450,0.462749,0.118619,0.534273,18.658432,33.557306,0.203015


In [23]:
mod_ids

model_id
StackedEnsemble_AllModels_AutoML_20210320_081445
StackedEnsemble_BestOfFamily_AutoML_20210320_081445
GLM_1_AutoML_20210320_081445
XGBoost_2_AutoML_20210320_081445
XGBoost_3_AutoML_20210320_081445
XGBoost_1_AutoML_20210320_081445
GLM_1_AutoML_20210320_100057
XGBoost_3_AutoML_20210320_100057
XGBoost_1_AutoML_20210320_100057
XGBoost_2_AutoML_20210320_100057


In [8]:
clean['wardsPlaced - SUPPB'].leaderboard

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
StackedEnsemble_AllModels_AutoML_20210412_055215,0.842118,0.462377,0.898576,0.260253,0.386511,0.14939
StackedEnsemble_BestOfFamily_AutoML_20210412_055215,0.841052,0.46488,0.898984,0.278232,0.387558,0.150201
XGBoost_grid__1_AutoML_20210412_055215_model_37,0.837346,0.463999,0.895238,0.269694,0.388164,0.150671
XGBoost_grid__1_AutoML_20210412_055215_model_18,0.837203,0.463242,0.893858,0.267328,0.387793,0.150383
XGBoost_grid__1_AutoML_20210412_055215_model_21,0.836588,0.463106,0.891627,0.272803,0.387301,0.150002
XGBoost_grid__1_AutoML_20210412_055215_model_26,0.836384,0.464614,0.892613,0.292495,0.387993,0.150538
GBM_grid__1_AutoML_20210412_055215_model_9,0.83535,0.468509,0.893514,0.268378,0.38939,0.151624
GBM_grid__1_AutoML_20210412_055215_model_6,0.834143,0.469578,0.893703,0.300922,0.390362,0.152383
GBM_grid__1_AutoML_20210412_055215_model_10,0.833779,0.469924,0.890935,0.282375,0.390216,0.152269
GBM_grid__1_AutoML_20210412_055215_model_3,0.833032,0.47213,0.891625,0.275929,0.391343,0.153149


In [10]:
clean['wardsPlaced - SUPPR'].leaderboard

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
StackedEnsemble_BestOfFamily_AutoML_20210412_095218,0.851077,0.446811,0.916068,0.281968,0.378851,0.143528
StackedEnsemble_AllModels_AutoML_20210412_095218,0.850963,0.446557,0.915604,0.290332,0.379062,0.143688
XGBoost_grid__1_AutoML_20210412_095218_model_16,0.849121,0.440684,0.914226,0.297143,0.377582,0.142569
XGBoost_grid__1_AutoML_20210412_095218_model_8,0.848306,0.441839,0.914217,0.268486,0.378197,0.143033
XGBoost_grid__1_AutoML_20210412_095218_model_26,0.846381,0.443831,0.913084,0.266969,0.379247,0.143829
XGBoost_grid__1_AutoML_20210412_095218_model_45,0.846362,0.444186,0.912113,0.28741,0.379287,0.143859
XGBoost_grid__1_AutoML_20210412_095218_model_49,0.846257,0.44511,0.912644,0.258275,0.379919,0.144338
XGBoost_grid__1_AutoML_20210412_095218_model_48,0.843139,0.449715,0.910838,0.281716,0.382171,0.146055
GLM_1_AutoML_20210412_095218,0.842135,0.449651,0.910005,0.258275,0.382031,0.145948
GBM_grid__1_AutoML_20210412_095218_model_9,0.841546,0.45332,0.909729,0.285265,0.383329,0.146941


In [10]:
WOESUPPR['wardsPlaced - SUPPB'].leaderboard

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
GBM_grid__1_AutoML_20210407_222508_model_4,0.536998,0.66233,0.685633,0.5,0.481798,0.23213
GBM_grid__1_AutoML_20210407_222508_model_8,0.536098,0.660118,0.683784,0.5,0.480918,0.231282
GBM_1_AutoML_20210407_222508,0.530793,0.647935,0.676695,0.499167,0.476612,0.227159
StackedEnsemble_AllModels_AutoML_20210407_222508,0.530514,0.641701,0.680674,0.5,0.474086,0.224758
GBM_grid__1_AutoML_20210407_222508_model_5,0.529578,0.67225,0.681884,0.499761,0.485326,0.235542
StackedEnsemble_BestOfFamily_AutoML_20210407_222508,0.529214,0.641842,0.679256,0.5,0.47416,0.224828
GBM_grid__1_AutoML_20210407_222508_model_6,0.528307,0.689654,0.678468,0.5,0.490205,0.240301
DeepLearning_grid__1_AutoML_20210407_222508_model_1,0.526868,1.14327,0.681413,0.5,0.561421,0.315194
GBM_2_AutoML_20210407_222508,0.526792,0.651125,0.675394,0.499761,0.477956,0.228442
GBM_3_AutoML_20210407_222508,0.526449,0.653,0.677867,0.49822,0.478696,0.22915


In [10]:
WOE['wardsPlaced - TOPB'].leaderboard

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
DeepLearning_grid__3_AutoML_20210405_214331_model_1,0.506148,0.673293,0.400571,0.499802,0.489869,0.239972
DeepLearning_grid__3_AutoML_20210405_214331_model_6,0.505997,0.680283,0.399697,0.497048,0.492834,0.242886
DeepLearning_grid__1_AutoML_20210405_214331_model_6,0.50497,1.43654,0.39746,0.5,0.581504,0.338147
XGBoost_3_AutoML_20210405_214331,0.504833,0.709678,0.400063,0.499865,0.505013,0.255039
XGBoost_grid__1_AutoML_20210405_214331_model_15,0.504278,0.768218,0.398624,0.5,0.524472,0.275071
DeepLearning_grid__3_AutoML_20210405_214331_model_5,0.503615,0.688751,0.405802,0.5,0.497326,0.247333
XGBoost_grid__1_AutoML_20210405_214331_model_12,0.503209,0.782272,0.403715,0.5,0.527571,0.278331
DeepLearning_grid__3_AutoML_20210405_214331_model_2,0.502908,0.673482,0.399341,0.5,0.490076,0.240175
GBM_grid__1_AutoML_20210405_214331_model_17,0.502551,0.695401,0.401072,0.49973,0.499114,0.249115
DeepLearning_grid__2_AutoML_20210405_214331_model_3,0.501967,1.06702,0.39702,0.5,0.561863,0.31569


In [11]:
WOESUPPR['wardsPlaced - TOPB'].leaderboard

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
DeepLearning_grid__1_AutoML_20210405_222522_model_1,0.515583,0.705769,0.414785,0.5,0.503853,0.253868
XGBoost_2_AutoML_20210405_222522,0.513684,0.768329,0.404751,0.499864,0.524413,0.275009
StackedEnsemble_BestOfFamily_AutoML_20210405_222522,0.512736,0.672494,0.406032,0.5,0.489637,0.239744
DeepLearning_grid__2_AutoML_20210405_222522_model_6,0.507228,0.687145,0.403799,0.5,0.496814,0.246824
GBM_grid__1_AutoML_20210405_222522_model_6,0.505916,0.679886,0.399893,0.499388,0.493093,0.24314
XGBoost_grid__1_AutoML_20210405_222522_model_12,0.50574,0.717277,0.4062,0.499591,0.508556,0.25863
StackedEnsemble_AllModels_AutoML_20210405_222522,0.503364,0.673189,0.399074,0.499321,0.489978,0.240078
DeepLearning_grid__2_AutoML_20210405_222522_model_2,0.502974,0.89604,0.393328,0.5,0.554037,0.306957
XGBoost_grid__1_AutoML_20210405_222522_model_21,0.50286,0.764169,0.399233,0.499457,0.523864,0.274433
GBM_grid__1_AutoML_20210405_222522_model_8,0.502828,0.685187,0.398807,0.499864,0.495318,0.24534


In [6]:
var_obj = 'wardsPlaced - JGB'
dfWOESUPPR = h2o.import_file('2SUPPRFinalDataframeWOE'+ var_obj +'.csv')
dfWOESUPPR = dfWOESUPPR.drop("C1")


Parse progress: |█████████████████████████████████████████████████████████| 100%


In [8]:
dfWOESUPPR.describe()

Rows:8111
Cols:70




,baron,dragon,goldDiff - TOP,goldDiff - JG,goldDiff - MID,goldDiff - ADC,goldDiff - SUPP,item0 - TOPB,item0 - SUPPR,item0 - JGB,item0 - MIDB,item0 - ADCB,item0 - SUPPB,item0 - TOPR,item0 - JGR,item0 - MIDR,item0 - ADCR,item1 - SUPPR,item1 - MIDB,item1 - ADCB,item1 - SUPPB,item1 - TOPR,level - TOPB,level - SUPPR,level - JGB,level - MIDB,level - ADCB,level - SUPPB,level - TOPR,level - JGR,level - MIDR,level - ADCR,minute,positionX - TOPB,positionX - SUPPR,positionX - JGB,positionX - MIDB,positionX - ADCB,positionX - SUPPB,positionX - TOPR,positionX - JGR,positionX - MIDR,positionX - ADCR,positionY - TOPB,positionY - SUPPR,positionY - JGB,positionY - MIDB,positionY - ADCB,positionY - SUPPB,positionY - TOPR,positionY - JGR,positionY - MIDR,positionY - ADCR,rift,wardsKilled - TOPB,wardsKilled - SUPPR,wardsKilled - JGB,wardsKilled - MIDB,wardsKilled - ADCB,wardsKilled - SUPPB,wardsKilled - TOPR,wardsKilled - JGR,wardsKilled - MIDR,wardsKilled - ADCR,xpDiff - TOP,xpDiff - JG,xpDiff - MID,xpDiff - ADC,xpDiff - SUPP,wardsPlaced - JGB
type,enum,enum,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,int,int,int,int,int,enum
mins,,,-7139.0,-5497.0,-7908.0,-14382.0,-4985.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,114.0,745.0,358.0,351.0,221.0,181.0,546.0,578.0,701.0,743.0,322.0,624.0,361.0,293.0,158.0,284.0,608.0,558.0,579.0,632.0,,,,,,,,,,,,-10760.0,-8889.0,-7826.0,-12631.0,-4588.0,
mean,,,75.32363457033661,19.652200715078273,-21.645049932190787,-65.10935766243377,84.64887190235488,3143.7218592035506,3221.4232523733203,2011.338922450992,2907.9226975711995,3185.2357292565653,3227.2020712612502,3100.4754037726543,2462.158796695845,2950.8831216865983,3178.6325977068177,697.7576131179878,482.16939958081616,380.6985575144866,696.889409443965,439.29256565158414,10.110960424115396,7.347675995561583,8.566884477869559,10.388238195043767,9.006041178646282,7.415361854271977,10.183947725311304,8.606583651830848,10.45949944519788,9.053137714215262,17.950930834668963,5443.952287017631,9621.629762051534,6045.62507705585,6048.657132289483,7452.676981876464,7300.2295647885585,7693.251633584022,8823.347675995561,8548.880902478117,9800.928862039207,7162.241647145851,7663.823079768216,6193.776969547529,6621.985451855505,5144.560350141783,5310.66378991493,9675.738626556527,8824.121193441006,8581.92158796696,7563.072494143756,,,,,,,,,,,,-127.37356676118846,-65.72025644186907,-107.6286524472938,-81.17753667858459,79.29281223030446,
maxs,,,7258.0,4915.0,6253.0,6750.0,5970.0,3364.0,3364.0,3364.0,3364.0,3364.0,3364.0,3364.0,3364.0,3364.0,3364.0,3364.0,3364.0,3364.0,3364.0,3364.0,18.0,17.0,18.0,18.0,18.0,18.0,18.0,18.0,18.0,18.0,52.0,14264.0,14572.0,14216.0,14171.0,14330.0,14131.0,14486.0,14506.0,14355.0,14568.0,14139.0,14658.0,14054.0,14284.0,14102.0,14104.0,14511.0,14529.0,14400.0,14656.0,,,,,,,,,,,,6501.0,6619.0,6605.0,6278.0,6910.0,
sigma,,,1499.5942741653985,1049.9605009221527,1306.874557631039,1455.7723930680938,856.4560320843156,771.6180619984633,617.6274826073055,1477.5148275934805,1070.955149848881,683.4036139368924,616.6519233332365,841.5961485728222,1367.5957858495588,1028.3405103985149,703.1727759176681,1018.153479806626,941.4558225172003,859.8733038304746,1024.7897639977714,866.8647012091508,4.7703633936327625,3.559145384551521,4.003073190133789,4.850596947053312,4.619487803719262,3.6014390518940806,4.824561466163897,4.058779107839974,4.885298986337698,4.678440433352843,10.616556820670954,3982.363610388109,3311.810405863629,3157.7314900280217,3169.892196945675,3806.9045922588266,3777.790675054864,4309.353116855005,3101.701730995166,3123.228876998903,3391.0591730913993,4367.125620530726,3732.4288870232353,3177.3621171685013,3193.392028104975,3426.4299791831736,3350.0702201477034,3988.14

In [20]:
dfClean.columns

['baron',
 'championCSD - TOPB',
 'championCSD - SUPPR',
 'championCSD - JGB',
 'championCSD - MIDB',
 'championCSD - ADCB',
 'championCSD - SUPPB',
 'championCSD - TOPR',
 'championCSD - JGR',
 'championCSD - MIDR',
 'championCSD - ADCR',
 'championGD - TOPB',
 'championGD - SUPPR',
 'championGD - JGB',
 'championGD - MIDB',
 'championGD - ADCB',
 'championGD - SUPPB',
 'championGD - TOPR',
 'championGD - JGR',
 'championGD - MIDR',
 'championGD - ADCR',
 'championId - TOPB',
 'championId - SUPPR',
 'championId - JGB',
 'championId - MIDB',
 'championId - ADCB',
 'championId - SUPPB',
 'championId - TOPR',
 'championId - JGR',
 'championId - MIDR',
 'championId - ADCR',
 'championXPD - TOPB',
 'championXPD - SUPPR',
 'championXPD - JGB',
 'championXPD - MIDB',
 'championXPD - ADCB',
 'championXPD - SUPPB',
 'championXPD - TOPR',
 'championXPD - JGR',
 'championXPD - MIDR',
 'championXPD - ADCR',
 'dead - TOPB',
 'dead - SUPPR',
 'dead - JGB',
 'dead - MIDB',
 'dead - ADCB',
 'dead - S

In [9]:
(1).astype(bool)

AttributeError: 'int' object has no attribute 'astype'